In [1]:
from typing import Optional, Iterable
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import jsonlines
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig
from transformers import (
    LlamaForCausalLM, CodeLlamaTokenizer,
    BitsAndBytesConfig, TrainingArguments)
import torch
import argparse
from datasets import Dataset

from data_management.lm_example import LmExample
from data_management.create_lm_dataset import split_file_path
from data_management.split_raw_data import TRAIN_NAME, VAL_NAME

/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_PATH = "/home/ubuntu/coq-modeling/data/data-points-partial-split"

MODEL_NAME = "codellama/CodeLlama-7b-hf"
OUTPUT_LOC = "/home/ubuntu/coq-modeling/models/codellama-7b-hf-test"
TRAIN_BATCH_SIZE = 4
LEARNING_RATE = 1.41e-5
NUM_TRAIN_EPOCHS = 1
MAX_STEPS = -1
PEFT_LORA_R = 64
PEFT_LORA_ALPHA = 16
MAX_SEQ_LEN = 512
GRADIENT_ACCUMULATION_STEPS = 2

DEFAULT_LOGGING_STEPS = 1
DEFAULT_SAVE_STEPS = 100
DEFAULT_SAVE_TOTAL_LIMIT = 10
DEFAULT_PUSH_TO_HUB = False
DEFAULT_HUB_MODEL_ID: Optional[str] = None
DEFAULT_LOG_WITH: Optional[str] = None


In [3]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=quantization_config,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.68s/it]


In [4]:
training_args = TrainingArguments(
    output_dir=OUTPUT_LOC,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    logging_steps=DEFAULT_LOGGING_STEPS,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    max_steps=MAX_STEPS,
    report_to=DEFAULT_LOG_WITH,
    save_steps=DEFAULT_SAVE_STEPS,
    save_total_limit=DEFAULT_SAVE_TOTAL_LIMIT,
    push_to_hub=DEFAULT_PUSH_TO_HUB,
    hub_model_id=DEFAULT_HUB_MODEL_ID,
)

In [5]:
def dataset_gen(dataset_path: str, split: str) -> Iterable[dict[str, str]]:
    file_path = split_file_path(dataset_path, split)
    with jsonlines.open(file_path, "r") as fin:
        for obj in fin:
            yield obj 

train_kwargs = {
    "dataset_path": DATA_PATH,
    "split": TRAIN_NAME,
}
val_kwargs = {
    "dataset_path": DATA_PATH,
    "split": VAL_NAME,
}
train_dataset = Dataset.from_generator(dataset_gen, gen_kwargs=train_kwargs)
val_dataset = Dataset.from_generator(dataset_gen, gen_kwargs=val_kwargs)

In [6]:
response_template = " <TACTIC>"
def formatting_examples_func(examples: list[dict[str, str]]) -> list[str]: 
    output_strs: list[str] = []
    for i in range(len(examples["input"])):
        example_in = examples["input"][i]
        example_out = examples["output"][i]
        collated_str = f"{example_in}\n{response_template}{example_out}"
        output_strs.append(collated_str) 
    return output_strs

In [7]:
tokenizer = CodeLlamaTokenizer.from_pretrained(MODEL_NAME)
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

peft_config = LoraConfig(
    r=PEFT_LORA_R,
    lora_alpha=PEFT_LORA_ALPHA,
    bias="none",
    task_type="CAUSAL_LM",
) 

trainer = SFTTrainer(
    model=model,
    args=training_args,
    max_seq_length=MAX_SEQ_LEN,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_examples_func,
    data_collator=collator,
    peft_config=peft_config
)

/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [9]:
trainer.train()

/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/trl/trainer/utils.py:110: UserWarning: Could not find response key ` <TACTIC>` in the following instance: <s> Lemma skipn_firstn: forall {A} n m (xs: list A), skipn n (firstn m xs) = firstn (m-n) (skipn n xs).
revert xs n; induction m; intros.
simpl.  destruct n; reflexivity.
destruct xs; simpl.
rewrite skipn_nil.
destruct match n with
            | O => S m
            | S l => (m - l)%nat
            end; reflexivity.
destruct n.
simpl.  reflexivity.
simpl.
apply IHm.<THM-SEP>A: Type
m: nat
IHm: forall (xs0 : list A) (n : nat),
skipn n (firstn m xs0) = firstn (m - n) (skipn n xs0)
a: A
xs: list A

skipn 0 (a :: firstn m xs) = firstn (S m) (skipn 0 (a :: xs))<GOAL-SEP>A: Type
m: nat
IHm: forall (xs0 : list A) (n0 : nat),
skipn n0 (firstn m xs0) = firstn (m - n0) (skipn n0 xs0)
a: A
xs: list A
n: nat

skipn (S n) (a :: firstn m xs) = firstn (m - n) (skipn (S n) (a :: xs))
 <TACTIC>
simpl.  This instance will be ignored in loss

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000


/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/trl/trainer/utils.py:110: UserWarning: Could not find response key ` <TACTIC>` in the following instance: <s> Theorem negb_involutive : forall b : bool, negb (negb b) = b. destruct b eqn:E.
  -  reflexivity.
  -  reflexivity. <THM-SEP>b: bool
E: b = false

negb (negb false) = false
 <TACTIC> reflexivity. </s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

KeyboardInterrupt: 